<a href="https://colab.research.google.com/github/duketran1996/NYC-Crime/blob/main/nyc_crime_data_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install openclean.**

In [ ]:
pip install openclean-core

**Loading the data file using Socrata. The file is NYPD Arrest Data (Year to Date) as listed below.**



In [ ]:
import gzip
import os

from openclean.data.source.socrata import Socrata

datafile = './uip8-fykc.tsv.gz'

# Download file only if it does not exist already.
#if not os.path.isfile(datafile):
with gzip.open(datafile, 'wb') as f:
  ds = Socrata().dataset('uip8-fykc')
  print('Downloading ...\n')
  print(ds.name + '\n')
  print(ds.description)
  ds.write(f)

ds


NYPD Arrest Data (Year to Date)

This is a breakdown of every arrest effected in NYC by the NYPD during the current year.
 This data is manually extracted every quarter and reviewed by the Office of Management Analysis and Planning. 
 Each record represents an arrest effected in NYC by the NYPD and includes information about the type of crime, the location and time of enforcement. 
In addition, information related to suspect demographics is also included. 
This data can be used by the public to explore the nature of police enforcement activity. 
Please refer to the attached data footnotes for additional information about this dataset.


**Important import. Run before executing the rest**

In [ ]:
from openclean.cluster.knn import knn_clusters, knn_collision_clusters
from openclean.function.similarity.base import SimilarityConstraint
from openclean.function.similarity.text import LevenshteinDistance
from openclean.function.token.ngram import NGrams
from openclean.function.value.threshold import GreaterThan
from openclean.operator.transform.update import update

**Data study: List number of columns.**

In [ ]:
from openclean.pipeline import stream

ds = stream(datafile)


print('Schema\n------')
for col in ds.columns:
    print("  '{}'".format(col))
    
print('\n{} rows.'.format(ds.count()))
print("There are {} rows and {} columns in the dataset.".format(ds.count(),len(ds.columns)))

Schema
------
  'ARREST_KEY'
  'ARREST_DATE'
  'PD_CD'
  'PD_DESC'
  'KY_CD'
  'OFNS_DESC'
  'LAW_CODE'
  'LAW_CAT_CD'
  'ARREST_BORO'
  'ARREST_PRECINCT'
  'JURISDICTION_CODE'
  'AGE_GROUP'
  'PERP_SEX'
  'PERP_RACE'
  'X_COORD_CD'
  'Y_COORD_CD'
  'Latitude'
  'Longitude'
  'New Georeferenced Column'

140413 rows.
There are 140413 rows and 19 columns in the dataset.


**Data study: Profile a sample of 10000 data to detect issues.**

In [ ]:
from openclean.profiling.column import DefaultColumnProfiler

profiles = ds.sample(n=10000, random_state=42).profile(default_profiler=DefaultColumnProfiler)

In [ ]:
profiles.stats()

,total,empty,distinct,uniqueness,entropy
ARREST_KEY,10000,0,10000,1.000000,13.287712
ARREST_DATE,10000,0,366,0.036600,8.396583
PD_CD,10000,1,166,0.016602,5.380796
PD_DESC,10000,2,158,0.015803,5.349247
KY_CD,10000,2,59,0.005901,4.501480
OFNS_DESC,10000,2,51,0.005101,4.290129
LAW_CODE,10000,0,421,0.042100,6.245291
LAW_CAT_CD,10000,108,4,0.000404,1.043776
ARREST_BORO,10000,0,5,0.000500,2.166298
ARREST_PRECINCT,10000,0,77,0.007700,6.113753


**Data study: Perform a scan to check date format. No issues found.**

In [ ]:
date = ds.distinct('ARREST_DATE')

import datetime 
def validate(date_text):
    try:
        datetime.datetime.strptime(date_text, '%m/%d/%Y')
    except ValueError:
        print(date_text)
        #raise ValueError("Incorrect data format, should be YYYY-MM-DD")

for i in date:
  validate(i)

**Convert to data frame for fixing issues.**

In [ ]:
fix = ds.to_df()

**Data Issues: In OFNS_DESC column, there are many repeated and miss spelling that needs to merge and fix. The impact with this change is later we would like to catergories offenses and have statistics on it.**

In [ ]:
offense = ds.select('OFNS_DESC').distinct()

clusters = knn_clusters(
  values=offense,
  sim=SimilarityConstraint(func=LevenshteinDistance(), pred=GreaterThan(0.7)),
  tokenizer=NGrams(n=4),
  minsize=2
)

for i in clusters:
  print(i)

Cluster({'INTOXICATED & IMPAIRED DRIVING': 2299, 'INTOXICATED/IMPAIRED DRIVING': 346})
Cluster({'KIDNAPPING & RELATED OFFENSES': 44, 'KIDNAPPING AND RELATED OFFENSES': 1})
Cluster({'OFFENSES AGAINST PUBLIC ADMINI': 4925, 'OFFENSES AGAINST PUBLIC SAFETY': 84})
Cluster({'OTHER STATE LAWS (NON PENAL LA': 536, 'OTHER STATE LAWS (NON PENAL LAW)': 5})
Cluster({'ADMINISTRATIVE CODE': 126, 'ADMINISTRATIVE CODES': 1})


**Data issues: Show KIDNAPPING spellings.**

In [ ]:
offense = ds.select(['OFNS_DESC']).distinct()

kid_val = []

for i in offense:
  if 'KIDNAPPING' in i:
    kid_val.append(i)
    print(i)

KIDNAPPING & RELATED OFFENSES
KIDNAPPING AND RELATED OFFENSES
KIDNAPPING


**Data fix: Change KIDNAPPING and KIDNAPPING & RELATED OFFENSES to KIDNAPPING AND RELATED OFFENSES**

In [ ]:
kid_dict = {
    'KIDNAPPING & RELATED OFFENSES': 'KIDNAPPING AND RELATED OFFENSES',
    'KIDNAPPING': 'KIDNAPPING AND RELATED OFFENSES'
}

fix = update(fix, columns='OFNS_DESC', func=kid_dict)

**Data fixed test: Test KIDNAPPING spellings**

In [ ]:
check_fix = fix.loc[fix['OFNS_DESC'].isin(kid_val)]
assert all(check_fix['OFNS_DESC'] == 'KIDNAPPING AND RELATED OFFENSES'), "KIDNAPPING spelling is not fixed: " + check_fix['OFNS_DESC'].unique()
print("Successfully fixed: " + check_fix['OFNS_DESC'].unique())

['Successfully fixed: KIDNAPPING AND RELATED OFFENSES']


**Data issues: Show ADMINISTRATIVE spellings.**

In [ ]:
offense = ds.select('OFNS_DESC').distinct()
administrative_val = []
for i in offense:
  if 'ADMINISTRATIVE' in i:
    administrative_val.append(i)
    print(i)

ADMINISTRATIVE CODES
ADMINISTRATIVE CODE


**Data fix: Change ADMINISTRATIVE CODES to ADMINISTRATIVE CODE**

In [ ]:
ad_dict = {
    'ADMINISTRATIVE CODES': 'ADMINISTRATIVE CODE',
}

fix = update(fix, columns='OFNS_DESC', func=ad_dict)

**Data fixed test: Test ADMINSTRATIVE spellings**

In [ ]:
check_fix = fix.loc[fix['OFNS_DESC'].isin(administrative_val)]
assert all(check_fix['OFNS_DESC'] == 'ADMINISTRATIVE CODE'), "ADMINISTRATIVE spelling is not fixed: " + check_fix['OFNS_DESC'].unique()
print("Successfully fixed: " + check_fix['OFNS_DESC'].unique())

['Successfully fixed: ADMINISTRATIVE CODE']


**Data issues: Show INTOXICATED spellings.**

In [ ]:
offense = ds.select('OFNS_DESC').distinct()
tox_val = []
for i in offense:
  if 'INTOXICATED' in i:
    tox_val.append(i)
    print(i)

INTOXICATED & IMPAIRED DRIVING
INTOXICATED/IMPAIRED DRIVING


**Data fix: Change INTOXICATED & IMPAIRED DRIVING, INTOXICATED/IMPAIRED DRIVING to INTOXICATED AND IMPAIRED DRIVING**

In [ ]:
tox_dict = {
    'INTOXICATED & IMPAIRED DRIVING': 'INTOXICATED AND IMPAIRED DRIVING',
    'INTOXICATED/IMPAIRED DRIVING': 'INTOXICATED AND IMPAIRED DRIVING'
}

fix = update(fix, columns='OFNS_DESC', func=tox_dict)

**Data fixed test: Test INTOXICATED spellings**

In [ ]:
check_fix = fix.loc[fix['OFNS_DESC'].isin(['INTOXICATED AND IMPAIRED DRIVING'] + tox_val)]
assert all(check_fix['OFNS_DESC'] == 'INTOXICATED AND IMPAIRED DRIVING'), "INTOXICATED spelling is not fixed: " + check_fix['OFNS_DESC'].unique()
print("Successfully fixed: " + check_fix['OFNS_DESC'].unique())

['Successfully fixed: INTOXICATED AND IMPAIRED DRIVING']


**Data issues: The borough of NYC that the arrest happen. The data K,M,B,Q,S is unclear to us.**

In [ ]:
print(fix['ARREST_BORO'].unique())

['B' 'M' 'Q' 'S' 'K']


**Data fix: Change ambiguous abbreviation of column ARREST_BORO to full form.**

In [ ]:
boro_dict = {
    'B': 'Bronx',
    'S': 'Staten Island',
    'K': 'Brooklyn',
    'M': 'Manhattan',
    'Q': 'Queens',
}

fix = update(fix, columns='ARREST_BORO', func=boro_dict)

**Data fixed test: Test ARREST_BORO fixed data**

In [ ]:
assert (sorted(fix['ARREST_BORO'].unique()) == sorted(boro_dict.values())), "ARREST_BORO is not fixed: " + fix['ARREST_BORO'].unique()
print("Successfully fixed: " + fix['ARREST_BORO'].unique())

['Successfully fixed: Bronx' 'Successfully fixed: Manhattan'
 'Successfully fixed: Queens' 'Successfully fixed: Staten Island'
 'Successfully fixed: Brooklyn']


**Data issues: The columns PERP_SEX and LAW_CAT_CD are also having values that is easier to read if written in full text instead of abbreviation.**

In [ ]:
sex = ds.distinct('PERP_SEX')

print(list(sex))

['M', 'F']


In [ ]:
law_cat_cd = ds.distinct('LAW_CAT_CD')

print(list(law_cat_cd))

['F', 'M', '', 'I', 'V']


**Data fix: Change abbreviation of LAW_CAT_CD to long form.**

In [ ]:
law_cat_cd_dict = {
    'F': 'Felony',
    'M': 'Misdemeanor',
    'V': 'Violation',
    'I': 'Traffic Infraction',
    '': 'Unknown'
}

fix = update(fix, columns='LAW_CAT_CD', func=law_cat_cd_dict)

**Data fixed test: Test LAW_CAT_CD fixed data**

In [ ]:
assert (sorted(fix['LAW_CAT_CD'].unique()) == sorted(law_cat_cd_dict.values())), "LAW_CAT_CD is not fixed: " + fix['LAW_CAT_CD'].unique()
print("Successfully fixed: " + fix['LAW_CAT_CD'].unique())

['Successfully fixed: Felony' 'Successfully fixed: Misdemeanor'
 'Successfully fixed: Unknown' 'Successfully fixed: Traffic Infraction'
 'Successfully fixed: Violation']


**Data fix: Change abbreviation of PERP_SEX to long form.**

In [ ]:
perp_sex_dict = {
    'F': 'Female',
    'M': 'Male'
}

fix = update(fix, columns='PERP_SEX', func=perp_sex_dict)

**Data fixed test: Test PERP_SEX fixed data**

In [ ]:
assert (sorted(fix['PERP_SEX'].unique()) == sorted(perp_sex_dict.values())), "PERP_SEX is not fixed: " + fix['PERP_SEX'].unique()
print("Successfully fixed: " + fix['PERP_SEX'].unique())

['Successfully fixed: Male' 'Successfully fixed: Female']


**Data issues: There are unnecessary columns in our dataset that we don't care about such as X_COORD_CD and Y_COORD_CD which list midblock X and Y-coordinate for New York State Plane Coordinate System, Long Island Zone, NAD 83, units feet (FIPS 3104)**

In [ ]:
display = ds.select(['X_COORD_CD','Y_COORD_CD']).to_df()

display.head()

,X_COORD_CD,Y_COORD_CD
0,1007453,233952
1,1001456,247485
2,1028605,187930
3,1039602,190480
4,949767,170539


**Data fix: Our solution is to drop the columns.**

In [ ]:
fix  = fix.drop(columns=['X_COORD_CD', 'Y_COORD_CD'])

**Data fixed test: Test X_COORD_CD and YCOORD_CD dropped column**

In [ ]:
assert (any(i not in fix.columns.values.tolist() for i in ['X_COORD_CD', 'Y_COORD_CD'])), "X_COORD_CD and Y_COORD_CD are not dropped"
print("Successfully dropped: " + str(fix.columns.values.tolist()))

Successfully dropped: ['ARREST_KEY', 'ARREST_DATE', 'PD_CD', 'PD_DESC', 'KY_CD', 'OFNS_DESC', 'LAW_CODE', 'LAW_CAT_CD', 'ARREST_BORO', 'ARREST_PRECINCT', 'JURISDICTION_CODE', 'AGE_GROUP', 'PERP_SEX', 'PERP_RACE', 'Latitude', 'Longitude', 'New Georeferenced Column']


**Data issues: The data ASIAN / PACIFIC ISLANDER is better to be fix by removing space between / for easier comparison for analysis later on.**

In [ ]:
race = ds.distinct('PERP_RACE')

for i in race:
  print(i)

BLACK
BLACK HISPANIC
WHITE
WHITE HISPANIC
ASIAN / PACIFIC ISLANDER
UNKNOWN
AMERICAN INDIAN/ALASKAN NATIVE


**Data fix: Remove space between ASIAN / PACIFIC ISLANDER.**

In [ ]:
race_dict = {
    'ASIAN / PACIFIC ISLANDER': 'ASIAN/PACIFIC ISLANDER'
}

fix = update(fix, columns='PERP_RACE', func=race_dict)

**Data fixed test: Test PERP_RACE fixed data**

In [ ]:
assert ('ASIAN / PACIFIC ISLANDER' not in fix['PERP_RACE'].unique()), "ASIAN / PACIFIC ISLANDER is not fixed"
print("Successfully fixed: " + fix['PERP_RACE'].unique())

['Successfully fixed: BLACK' 'Successfully fixed: BLACK HISPANIC'
 'Successfully fixed: WHITE' 'Successfully fixed: WHITE HISPANIC'
 'Successfully fixed: ASIAN/PACIFIC ISLANDER'
 'Successfully fixed: UNKNOWN'
 'Successfully fixed: AMERICAN INDIAN/ALASKAN NATIVE']


**Data issues: Found new issues with PD_DESC. Some spellings are incorrect. This also impacts as we want to catergorize the PD description to compare with the offense description.**

In [ ]:
pd = ds.select('PD_DESC').distinct()

clusters = knn_clusters(
  values=pd,
  sim=SimilarityConstraint(func=LevenshteinDistance(), pred=GreaterThan(0.9)),
  tokenizer=NGrams(n=4),
  minsize=2
)

for i in clusters:
  print(i)

Cluster({'PROSTITUTION 2, UNDER 16': 4, 'PROSTITUTION 1, UNDER 11': 1})
Cluster({'NY STATE LAWS,UNCLASSIFIED MIS': 428, 'NY STATE LAWS,UNCLASSIFIED VIO': 262})
Cluster({'TRESPASS 3, CRIMINAL': 1049, 'TRESPASS 2, CRIMINAL': 503})
Cluster({'UNAUTHORIZED USE VEHICLE 3': 522, 'UNAUTHORIZED USE VEHICLE 2': 241})
Cluster({'RECKLESS ENDANGERMENT 1': 647, 'RECKLESS ENDANGERMENT 2': 608})
Cluster({'KIDNAPPING 2': 9, 'KIDNAPPING 1': 1})
Cluster({'CONTROLLED SUBSTANCE, POSSESSI': 4773, 'CONTROLLED SUBSTANCE,POSSESS.': 474})
Cluster({'CUSTODIAL INTERFERENCE 2': 10, 'CUSTODIAL INTERFERENCE 1': 5})
Cluster({'CONTROLLED SUBSTANCE,INTENT TO': 2400, 'CONTROLLED SUBSTANCE, INTENT T': 362})
Cluster({'CONTROLLED SUBSTANCE,SALE 3': 976, 'CONTROLLED SUBSTANCE, SALE 5': 67, 'CONTROLLED SUBSTANCE,SALE 2': 66, 'CONTROLLED SUBSTANCE,SALE 1': 56, 'CONTROLLED SUBSTANCE, SALE 4': 23})
Cluster({'SALE SCHOOL GROUNDS': 40, 'SALE SCHOOL GROUNDS 4': 1})
Cluster({'AGGRAVATED HARASSMENT 2': 2365, 'AGGRAVATED HARASSMENT 1

**Only miss spelling one is considered to be fixed such as: CONTROLLED SUBSTANCE, POSSESSI, CONTROLLED SUBSTANCE, INTENT T, CONTROLLED SUBSTANCE,POSSESS., and spaces between CONTROLLED SUBSTANCE, SALE**

In [ ]:
pd_desc = ds.select('PD_DESC').distinct()

for i in pd_desc:
  if 'CONTROLLED SUBSTANCE,' in i:
    print(i)

CONTROLLED SUBSTANCE, POSSESSI
CONTROLLED SUBSTANCE,INTENT TO
CONTROLLED SUBSTANCE, INTENT T
CONTROLLED SUBSTANCE,SALE 3
CONTROLLED SUBSTANCE, SALE 5
CONTROLLED SUBSTANCE,POSSESS.
CONTROLLED SUBSTANCE,SALE 1
CONTROLLED SUBSTANCE, SALE 4
CONTROLLED SUBSTANCE,SALE 2


**Data fix: Change spacing in SALE and spelling to POSSESSION and INTENT**



In [ ]:
control_dict = {
    'CONTROLLED SUBSTANCE, POSSESSI': 'CONTROLLED SUBSTANCE, POSSESSION',
    'CONTROLLED SUBSTANCE,POSSESS.': 'CONTROLLED SUBSTANCE, POSSESSION',
    'CONTROLLED SUBSTANCE,INTENT TO': 'CONTROLLED SUBSTANCE, INTENT',
    'CONTROLLED SUBSTANCE, INTENT T': 'CONTROLLED SUBSTANCE, INTENT',
    'CONTROLLED SUBSTANCE,SALE 3': 'CONTROLLED SUBSTANCE, SALE 3',
    'CONTROLLED SUBSTANCE,SALE 1': 'CONTROLLED SUBSTANCE, SALE 1',
    'CONTROLLED SUBSTANCE,SALE 2': 'CONTROLLED SUBSTANCE, SALE 2'
}

fix = update(fix, columns='PD_DESC', func=control_dict)

**Data fixed test: Test PD_DESC fixed data**

In [ ]:
check_fix = fix[fix['PD_DESC'].str.contains('CONTROLLED SUBSTANCE')]
assert (any(i not in check_fix['PD_DESC'].unique() for i in control_dict.keys())), "PD_DESC is not fixed: " + check_fix['PD_DESC'].unique()
print("Successfully fixed: " + check_fix['PD_DESC'].unique())

['Successfully fixed: CONTROLLED SUBSTANCE, POSSESSION'
 'Successfully fixed: CONTROLLED SUBSTANCE, INTENT'
 'Successfully fixed: CONTROLLED SUBSTANCE, SALE 3'
 'Successfully fixed: CONTROLLED SUBSTANCE, SALE 5'
 'Successfully fixed: CONTROLLED SUBSTANCE, SALE 1'
 'Successfully fixed: CONTROLLED SUBSTANCE, SALE 4'
 'Successfully fixed: CONTROLLED SUBSTANCE, SALE 2']


**Only fix DRUG spelling**

In [ ]:
pd_desc = ds.select('PD_DESC').distinct()

for i in pd_desc:
  if 'IMPAIRED DRIVING' in i:
    print(i)

IMPAIRED DRIVING,DRUG
IMPAIRED DRIVING / ALCOHOL
IMPAIRED DRIVING, DRUGS


**Data fix: Fix DRUG spelling.**

In [ ]:
impair_dict = {
    'IMPAIRED DRIVING, DRUGS': 'IMPAIRED DRIVING / DRUG',
    'IMPAIRED DRIVING,DRUG': 'IMPAIRED DRIVING / DRUG'
}

fix = update(fix, columns='PD_DESC', func=impair_dict)

**Data fixed test: Test PD_DESC fixed data**

In [ ]:
check_fix = fix[fix['PD_DESC'].str.contains('IMPAIRED DRIVING')]
assert (all(i not in check_fix['PD_DESC'].unique() for i in impair_dict.keys())), "PD_DESC is not fixed: " + check_fix['PD_DESC'].unique()
print("Successfully fixed: " + check_fix['PD_DESC'].unique())

['Successfully fixed: IMPAIRED DRIVING / DRUG'
 'Successfully fixed: IMPAIRED DRIVING / ALCOHOL']


**Finalize data set: Save data clean file to csv file for analysis.**

In [ ]:
fix.to_csv(r'arrest_data_clean.csv')